<a href="https://colab.research.google.com/github/cesarofuchi/Projects/blob/main/TabelaSanepar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Relatório com previsão de rodizio de água para sua região em Curitiba</h1>

A seca pelo qual Curitiba e região metropolitana esta passando pode estar longe de acabar. A Sanepar tem feito um esforço para divulgar o META 20 para economizarmos 20% de água e assim diminuirmos o tempo de rodízio. Os rodízios de água mudam de acordo com os níveis de água dos reservatórios. No meu condomínio o rodízio é avisado no dia por whatsapp, e se você for como eu, que não se informa como deveria, tem que ficar revisando as mensagens para ter certeza de quando começa e quando termina. 

Pelo site da [Sanepar](http://site.sanepar.com.br/mapa-rodizio-abastecimento-curitiba-rmc) é possível verificar pelo endereço como está a previsão do abastecimento em sua região. Porém ele só informa quando será o próximo rodízio.
 
![picture](https://github.com/cesarofuchi/Projects/blob/main/sanepar/img/banner_20porcento.jpg?raw=true)

## Obtendo os dados
Acontece que na [agência estadual de notícias](http://www.aen.pr.gov.br/)  existe uma [tabela](http://www.aen.pr.gov.br/arquivos/0910rodizioatualizada.pdf) mais detalhada, com uma previsão maior das datas de rodízio. 


![picture](https://github.com/cesarofuchi/Projects/blob/main/sanepar/img/Tabela.JPG?raw=true)

A tabela possui as datas de início e término do rodízio, quais os bairros afetados e o Grupo do rodízio. Em Curitiba, atualmente temos 3 grupos, mas isso já mudou antes também. 

Um mesmo bairro pode ter dias diferentes de rodízio, então é preciso se informar qual o seu reservatório de abastecimento, fazendo uma procura pelo bairro e pelas datas de início e término do rodízio (pode conferir pelo site da sanepar isso). 

Feito isso podemos usar o identificador da primeira coluna ou o nome da sua área (na penúltima coluna) para saber qual a sua micro-região. 

Exemplo: RISO - (Área do Recalque do Reservatório Iguaçu)

# Atualização 12/11

[tabela 12/11](http://www.aen.pr.gov.br/arquivos/1211rodizio.pdf)

- Nessa atualização foram retirados os códigos de cada região. Portanto devemos procurar apenas pela string da região.
- Tabelas separadas em uma mesma página: O código foi atualizado para que funcione com mais de  uma tabela por página


#Continuando ...
Munidos dessa informação vamos extrair as datas da sua região e gerar um relatório em PDF para você imprimir, deixar no elevador, na geladeira, ou simplemente deixar no celular para avaliar as datas em que faltará água na sua casa.

Primeiramente devemos fazer o download da tabela para deixar aqui na maquina virtual. Esse link irá mudar de tempos em tempo, então procure a mais atualizada.

Vou usar a biblioteca requests para salvar o arquivo na maquina local.

In [1]:
import requests
#import textract
response = requests.get("http://www.aen.pr.gov.br/arquivos/1211rodizio.pdf")
#myfile = open("out.pdf", "w")

with open('document.pdf', 'wb') as fw:
    fw.write(response.content)


A tabela está salva como "document.pdf"

## Extração dos dados da tabela em PDF


Para extrar e manipular os dados da tabela será utilizada a biblioteca de Python [PDF Plumber](https://github.com/jsvine/pdfplumber). Ela possui uma grande gama de funções e argumentos, e aqui nós vamos começar a nos aventurar com essas funções.

Lembrando: Nativamente, o Google Colab não possui essa biblioteca instalada, então nós precisamos instalar a mesma no nosso ambiente.

Para isso, vamos utilizar o gerenciador de pacotes pip, e com o comando install vamos instalar a biblioteca.

In [2]:
!pip install pdfplumber

     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 5.6MB 7.0MB/s 
     |████████████████████████████████| 143kB 40.5MB/s 
     |████████████████████████████████| 13.7MB 294kB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.24-cp36-none-any.whl size=31125 sha256=c6d56fe7eb17c5252d226be7d9edb40d46ea1dec7f67f28885131eb4320b1e9a
  Stored in directory: /root/.cache/pip/wheels/42/e7/9d/783e40e447a240ace74b520158a133175853f3c3b2999f4520
Successfully built pdfplumber


### pdfplumber.open()

Para abrir um arquivo, vamos associá-lo a uma variável, para que ele possa ser facilmente acessado depois.

Utilizamos o comando `pdfplumber.open('caminho/do/meu/arquivo')` com o caminho do arquivo dentro dos parênteses, dentro de aspas, simples ou duplas.

In [3]:
#biblioteca para manipular arquivos
import pdfplumber

In [4]:
#carregar a tabela em pdf
pdf = pdfplumber.open("document.pdf")

### pages
Para acessar as páginas existentes no arquivo, utilizamos a função `pages`.

Ela retorna uma lista com as páginas do arquivo.

Perceba que a tabela de rodízio possui muitas páginas

In [5]:
pdf.pages

[<Page:1>,
 <Page:2>,
 <Page:3>,
 <Page:4>,
 <Page:5>,
 <Page:6>,
 <Page:7>,
 <Page:8>,
 <Page:9>,
 <Page:10>,
 <Page:11>,
 <Page:12>,
 <Page:13>,
 <Page:14>]

In [6]:
pdf.metadata

{'Author': 'Edymilson Luiz Dos Santos',
 'CreationDate': "D:20201109091508-03'00'",
 'Creator': 'Microsoft® Excel® 2013',
 'ModDate': "D:20201109091508-03'00'",
 'Producer': 'Microsoft® Excel® 2013'}

### extract_text()
Para extrair um texto de um pdf utilizamos a função `extract_text`

Ela retorna o texto em formato string para buscar a informação. Em geral esse processa implica em separar as linhas, procurar por expressões regulares 'regex', etc.

In [7]:
# abrir a primeira pagina do documento
pagina = pdf.pages[0]
texto = pagina.extract_text()
texto

'10 de Novembro de 2020\nInício Normalização Bairros afetados Grupo\nCuritiba (área do Recalque Baixo do Reservatório Cajuru): Cabral, Juvevê, Hugo Lange, Alto da Glória, Centro, \n10/11/2020 16:00 12/11/2020 04:00 Jardim Social, Bacacheri, Alto da XV, Cristo Rei, Prado Velho, Jardim Botânico. 2\nCuritiba (área do Recalque Alto do Reservatório Cajuru): Agua Verde, Batel, Centro, Rebouças.\n10/11/2020 16:00 12/11/2020 04:00 2\nSão José dos Pinhais (área do Recalque do Reservatório São José Centro): São Domingos, Aristocrata, Aeroporto, \n10/11/2020 16:00 12/11/2020 04:00 Águas Belas, Centro. 2\nSão José dos Pinhais (Área do Reservatório São Marcos): São Marcos, Campina do Taquaral, Parque da Fonte.\n10/11/2020 16:00 12/11/2020 04:00 2\nPinhais (área do Recalque do Reservatório Jacob Macanhan): Alto Atuba 1 e 2, Emiliano Perneta 2, Centro 1 e 2, \nPalmital 1 e 2, Sete Vilas, Estância Pinhais 2.\n10/11/2020 16:00 12/11/2020 04:00 2\nPinhais (Área da Gravidade do Reservatório Apoiado Alpha

###extract_table()
Para o caso de planilhas em pdf (como é o caso da [tabela](http://www.aen.pr.gov.br/arquivos/0910rodizioatualizada.pdf) de rodízio) existe uma função mais apropriada chamada `extract_table`, que retorna os dados em formato de lista

In [8]:
pdf.pages[0].extract_table()

[['10 de Novembro de 2020', None, None, ''],
 ['Início', 'Normalização', 'Bairros afetados', 'Grupo'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'Curitiba (área do Recalque Baixo do Reservatório Cajuru): Cabral, Juvevê, Hugo Lange, Alto da Glória, Centro, \nJardim Social, Bacacheri, Alto da XV, Cristo Rei, Prado Velho, Jardim Botânico.',
  '2'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'Curitiba (área do Recalque Alto do Reservatório Cajuru): Agua Verde, Batel, Centro, Rebouças.',
  '2'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'São José dos Pinhais (área do Recalque do Reservatório São José Centro): São Domingos, Aristocrata, Aeroporto, \nÁguas Belas, Centro.',
  '2'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'São José dos Pinhais (Área do Reservatório São Marcos): São Marcos, Campina do Taquaral, Parque da Fonte.',
  '2'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'Pinhais (área do Recalque do Reservatório Jacob Macanhan): Alto Atuba 1 e 2, Emiliano Perneta 2,

###find_tables()

Existem casos em que existem planilhas separadas na mesma página. A função `extract_table` somente retorna a maior tabela da página. Portanto devemos procurar essas planilhas pela função `find_table` e procurar se existem mais de uma tabela dentro da página na variável `tablesPage`. Dessa forma adicionamos mais um nível de tabelas no loop.

In [9]:
tableTest=[] 
#exemplo em que existe mais de uma tabela 
tablesPage=pdf.pages[1].find_tables()
  #podem existir mais de uma tabela na mesma pagina
for j in range(len(tablesPage)):
  tableTest=tableTest+tablesPage[j].extract(x_tolerance = 1) 
tableTest

[['10/11/2020 16:00',
  '12/11/2020 04:00',
  'Almirante Tamandaré: Vila Sta. Terezinha, Jardim Colonial, Jardim São Domingos, Vila Rachel, Recantos dos \nPapagaios, Parte Alta Paraiso, Jardim Planalto, Jardim Valença Jardim Central , Mato Dentro, Jardim Dona \nBelezária, Cinthia Mara, Boixininga, Vila Pacotuba, Jardim Bela Vista, Planta Maria Carla, Planta Mosacal.',
  '1ALM'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'Bocaiuva do Sul: Vila Esperança, Vila Palmira e Jardim Lucy',
  '1BOS'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'Colombo (área da gravidade colombo sede): Butiatumirim, Fervida, São João, Santa Gema, Cercadinho, Serrinha, \nItajacuru, Centro (parcial).',
  '1CSE'],
 ['10/11/2020 16:00',
  '12/11/2020 04:00',
  'Colombo (área da válvula Pau Brasil): Parque Embu',
  '1CSE'],
 ['10/11/2020 09:00',
  '11/11/2020 15:00',
  'Quatro Barras: Santa Luzia',
  '1BOC'],
 [None, None, None, ''],
 ['11 de Novembro de 2020', None, None, ''],
 ['Início', 'Normalização', 'Ba

A próxima etapa será juntar todas as páginas para depois extrairmos todas as datas de rodízio para a nossa região. 
O código a seguir concatena essas listas na variável table

In [10]:
#iterar por todas as paginas e concatenar as tabelas
table=[]
tablesPage=[]         
for i in range(len(pdf.pages)):  
  pagina=pdf.pages[i]  
  tablesPage=pdf.pages[i].find_tables()
    #podem existir mais de uma tabela na mesma pagina
  for j in range(len(tablesPage)):
    table=table+tablesPage[j].extract(x_tolerance = 1) 
  

Para extrair somente as datas da região escolhida, podemos usar a primeira coluna que é o código `coluna=0` ou a sexta coluna `coluna=5` se usarmos toda a string da área/reservatório e comparar com a string da nossa região `str_procura`. É preciso descomentar qual o conjunto escolhido.

O código separa as datas de início e fim do rodizio da região na variável `resultado`

In [11]:
# escolher método de procura pela sua região

#procura por codigo
#str_procura='RBBC'
#coluna=0

#procura por área / reservatório
from datetime import datetime



weekday_name = ["SEG", "TER", "QUA", "QUI", "SEX", "SAB", "DOM"]


str_procura="área do Recalque Alto do Reservatório Cajuru"
coluna=-2

resultado=[]

for i in range(len(table)):
  if table[i][coluna] is not None:
    if table[i][coluna].find(str_procura)>=0:
      # resultado encontrado, separar os dados de inicio e fim do rodizio

      dt1=datetime.strptime(table[i][0], '%d/%m/%Y %H:%M')      
      dt2=datetime.strptime(table[i][1], '%d/%m/%Y %H:%M')      
      str1=weekday_name[dt1.weekday()]+' '+table[i][0]
      str2=weekday_name[dt2.weekday()]+' '+table[i][1]
      resultado.append([str1,str2])
      # salvando o local para o relatório
      textoLocal=table[i][coluna]

      print('inicio=',table[i][0], weekday_name[dt1.weekday()],'fim=', table[i][1], weekday_name[dt1.weekday()], textoLocal)
      
#adicionando nomes para as colunas
resultado.insert(0,['Início','Fim',])


inicio= 10/11/2020 16:00 TER fim= 12/11/2020 04:00 TER Curitiba (área do Recalque Alto do Reservatório Cajuru): Agua Verde, Batel, Centro, Rebouças.
inicio= 16/11/2020 16:00 SEG fim= 18/11/2020 04:00 SEG Curitiba (área do Recalque Alto do Reservatório Cajuru): Agua Verde, Batel, Centro, Rebouças.
inicio= 19/11/2020 16:00 QUI fim= 21/11/2020 04:00 QUI Curitiba (área do Recalque Alto do Reservatório Cajuru): Agua Verde, Batel, Centro, Rebouças.


In [12]:
resultado

[['Início', 'Fim'],
 ['TER 10/11/2020 16:00', 'QUI 12/11/2020 04:00'],
 ['SEG 16/11/2020 16:00', 'QUA 18/11/2020 04:00'],
 ['QUI 19/11/2020 16:00', 'SAB 21/11/2020 04:00']]

## Gerando um relatório em PDF
Com os resultados em mãos, vamos gerar um relatório em pdf para divulgar o resultado.

Para gerar um relatório em pdf será usada a biblioteca [reportlab](https://www.reportlab.com/). Vamos usar o mínimo de elementos nesse código, um texto, uma tabela e uma imagem. Para mais detalhes da biblioteca vejam o [manual de uso](https://www.reportlab.com/docs/reportlab-userguide.pdf). 

A biblioteca também não é padrão no Colab, então precisamos adiciona-la ao nosso projeto com `!pip install reportlab`






In [13]:
!pip install reportlab

     |████████████████████████████████| 2.6MB 13.4MB/s 


Adicionando elementos que vamos utilizar da biblioteca:

In [14]:
from reportlab.pdfgen import canvas
from reportlab.platypus import Frame, Image
from reportlab.lib.units import cm
from reportlab.lib import colors
from reportlab.platypus import Paragraph, Frame, Table, Spacer, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.styles import ParagraphStyle

### Entendendo a estrutura do reportlab utilizada
Para melhor entender a estrutura de código para gerar o relatório veja a figura a seguir.

O `Canvas` é uma tela A4 (com uma margem interna)

Vamos adicionar um `Frame` que é um container de elementos nesse `Canvas` a 3 cm de distância da margem esquerda e a 12 cm de distância da margem inferior. As distãncias são relativas a posição (0,0) no canto inferior esquedo.


Dentro do `Frame` montamos uma `Story`, que é uma sequência de elementos: 

- Parágrafo 1: texto padrão
- Parágrafo 2: criada a partir do texto da região procurada
- Tabela: criada a partir da tabela de resultados 
- Imagem: criada a partir de um link

Foram adicionados espaçamentos (`Spacer`) entre os elementos

![picture](https://github.com/cesarofuchi/Projects/blob/main/sanepar/img/reportlab.JPG?raw=true)







### Código
Deixei o código inteiro para que seja mais fácil editar as posições e estilos dos elementos, e gerar um pdf novo. Assim pode-se aprender na prática o que cada mudança nos elementos provoca.

Ao final do processo é gerado um pdf com informações sobre a sua região e o horário de início e fim do rodízio.

In [16]:
#criação do Canvas
nome_pdf = "rodizio_sanepar1211.pdf" 
c = canvas.Canvas(nome_pdf)
c.setTitle(nome_pdf)
table_size=len
#Criação da tabela a partir do resultado
rtable=Table(resultado)
 # propriedades da tabela
t_fontsize=16
t_leading=t_fontsize+2

rtable.setStyle(TableStyle([
  ('LEADING', (0, 0), (-1, -1), t_leading),
  ('SIZE', (0, 0), (-1, -1), t_fontsize),
  ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.black),
  ('BOX', (0, 0), (-1, -1), 0.25, colors.black)
]))
###############################################################
# padrão é formato A4 (21 cm)
# posição em CM, posição (0,0) canto inferior esquerdo

# estilo do paragrafo 1        
titleFormat = '<font size="18" name="Helvetica" color="rgb(0, 89, 164)"><b>%s</b></font>'
style = getSampleStyleSheet()['Normal']
style.leading = 24

# paragrafo 1
p1 = Paragraph(titleFormat % 'RODÍZIO DE ABASTECIMENTO', style)

# estilo do paragrafo 2
titleFormat = '<font size="16" name="Helvetica" color=black><i>%s</i></font>'
# paragrafo 2 
p2 = Paragraph(titleFormat % textoLocal, style)

# imagem 
im=Image("https://github.com/cesarofuchi/Projects/blob/main/sanepar/img/banner_20porcento.jpg?raw=true")
# restrição de tamanho para reduzir a imagem de forma proporcional
im._restrictSize(10 * cm, 12 * cm)
im.hAlign = 'RIGHT'

titleFormat = '<font size="16" name="Helvetica" color=black><b>%s</b></font>'
p3 = Paragraph(titleFormat % 'LEI Nº 20189/2020:', style)

imMask=Image("https://github.com/cesarofuchi/Projects/blob/main/sanepar/img/mascara.jpg?raw=true")
imMask._restrictSize(10* cm, 10 * cm)
imMask.hAlign = 'CENTER'


# story com os elementos espaçados        
story = [p1,
         Spacer(1, 10),
         p2,
         Spacer(1, 10),
         rtable,
         Spacer(1, 30),
         p3,
         Spacer(1, 5),
         imMask,
         Spacer(10, 40),
         im]

# criação do Frame
f = Frame(3*cm, 1*cm, 16 * cm, 25 * cm)
# adicionando o Frame (com Story) ao Canvas
f.addFromList(story,c)

# salvando no diretório local
c.save()
print('{}.pdf criado com sucesso!'.format(nome_pdf))

rodizio_sanepar1211.pdf.pdf criado com sucesso!
